In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import numpy as np

In [ ]:
# Configuration
num_classes = 6
num_epochs = 5
batch_size = 32
learning_rate = 1e-4
confidence_threshold = 0.6
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Data transforms
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
# Load datasets
train_dataset = datasets.ImageFolder("dataset_split/train", transform=train_transforms)
val_dataset = datasets.ImageFolder("dataset_split/val", transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
# Load model
model = models.resnet50(weights=None)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("resnet50_waste_classifier.pth"))

<All keys matched successfully>

In [11]:
# Freeze early layers except last block and fc
for name, param in model.named_parameters():
    if "layer4" not in name and "fc" not in name:
        param.requires_grad = False

model = model.to(device)

In [ ]:
# Loss and optimizer
class_weights = torch.tensor([1.0, 1.5, 1.0, 1.0, 1.0, 1.0]).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
# Fine-tuning loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    scheduler.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_dataset):.4f}")

Epoch 1/5, Loss: 0.4146
Epoch 2/5, Loss: 0.3235
Epoch 3/5, Loss: 0.2848
Epoch 4/5, Loss: 0.2627
Epoch 5/5, Loss: 0.2434


In [ ]:
# Save fine-tuned model
torch.save(model.state_dict(), "resnet50_waste_finetuned.pth")
print(" Fine-tuned model saved as resnet50_waste_finetuned.pth")

✅ Fine-tuned model saved as resnet50_waste_finetuned.pth


In [ ]:
# Validation & Confusion Matrix
model.eval()
all_preds, all_labels, all_probs = [], [], []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1)
        max_probs, preds = torch.max(probs, dim=1)

        preds = torch.where(max_probs < confidence_threshold,
                            torch.tensor(num_classes).to(device), preds)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(max_probs.cpu().numpy())

labels_for_cm = list(range(num_classes)) + [num_classes] 
cm = confusion_matrix(all_labels, all_preds, labels=labels_for_cm)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[132   0   2  11   1   3   8]
 [  0 143   4   1   2   1   6]
 [  0   2 140   2   4   0   9]
 [  3   0   2 140   1   1  10]
 [  0   5   3   0 141   4   5]
 [  0   1   2   5   2 142   5]
 [  0   0   0   0   0   0   0]]


In [ ]:
# test_resnet50_waste_input.py

import torch
from torchvision import models, transforms
from PIL import Image
import os

# Configuration
num_classes = 6
class_names = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]
confidence_threshold = 0.6
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load fine-tuned model
model = models.resnet50(weights=None)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("resnet50_waste_finetuned.pth", map_location=device))
model = model.to(device)
model.eval()

# Image preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Ask user for image path
image_path = input("Enter the full path to the image: ").strip()

if not os.path.exists(image_path):
    print("Error: File does not exist.")
    exit()

# Load and preprocess image
image = Image.open(image_path).convert("RGB")
input_tensor = preprocess(image).unsqueeze(0).to(device)  # add batch dimension

# Predict
with torch.no_grad():
    outputs = model(input_tensor)
    probs = torch.softmax(outputs, dim=1)
    max_prob, pred = torch.max(probs, dim=1)

    if max_prob.item() < confidence_threshold:
        predicted_label = "Not Recognized"
    else:
        predicted_label = class_names[pred.item()]

print(f"Predicted class: {predicted_label}, Confidence: {max_prob.item():.4f}")


Enter the full path to the image:  C:\Users\PC\Desktop\IMG_20250816_093237.jpg


✅ Predicted class: metal, Confidence: 0.9987
